In [ ]:
import pandas as pd
import skyfield
import re

from IPython.display import clear_output
from skyfield.api import EarthSatellite, utc, wgs84, load
from datetime import datetime, timedelta

In [ ]:
rootDir = '' #The file path to the root directory for your project
resultDir = rootDir + '05_result/'
logDir = rootDir + '06_log/'

In [ ]:
data = pd.read_excel(logDir + 'alouette_tle.xlsx')
data.head()

In [ ]:
tle = data.loc[:, ["EPOCH", "TLE_LINE1", "TLE_LINE2"]]
tle.EPOCH

In [ ]:
alouette = pd.read_csv(resultDir + 'result_master.csv')
alouette.head()

In [ ]:
# function for settling quadrant ambiguity (before I realized skyfield has NWES values available)

def quad_latlon(coords):
    splt_str = re.split('(\d+)', coords)
    new_coord = float(''.join(splt_str[1:-1]))
    
    quad = splt_str[-1:]
    
    if quad == ["N"]:
        return new_coord
    elif quad == ["S"]:
        return new_coord*-1
    elif quad == ["E"]:
        return new_coord
    elif quad == ["W"]:
        return new_coord*-1
    else:
        print(splt_str)
        print("An Error has Occured")
        return 0

In [ ]:
# function for getting the nearest TLE record

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [ ]:
result_dir = []
result_sub_dir = []
result_name = []

result_type = []

result_read_epoch = []
result_read_station = [] 
result_tle_epoch = []

result_flag = []

result_events = []
result_event_times = []

result_alt = []
result_distance =[] 

result_lat = []
result_lon = []
result_height = []

## len(alouette)

for i in range(len(alouette)):
    record_epoch = alouette.loc[i, "Timestamp"]
    record_epoch = datetime.strptime(record_epoch, '%Y-%m-%d %H:%M:%S')
    station = alouette.iloc[i, -2:]
    quality = alouette.loc[i, "time_quality"]
    meta_type = alouette.loc[i, "processed_image_class"]

    nearest_epoch = nearest(tle.EPOCH, record_epoch)
    nearest_tle = tle.loc[tle['EPOCH'] == nearest_epoch]
    
    station = wgs84.latlon(quad_latlon(station[0]), quad_latlon(station[1]))
    
    ts = load.timescale()
    satellite = EarthSatellite(nearest_tle.iat[0,1], nearest_tle.iat[0,2], 'Alouette-1', ts)
    
    d = record_epoch.replace(tzinfo=utc)
    t = ts.utc(d)
    
    geocentric = satellite.at(t)
    lat, lon = wgs84.latlon_of(geocentric)
    height = wgs84.height_of(geocentric)
    
#    difference between station
    difference = satellite - station

    topocentric = difference.at(t)
    alt, az, distance = topocentric.altaz()
    
    if alt.degrees > 0:
        flag = 0 
    else:
        flag = 1
    
    if quality == 1:
        t0 = t - 0.010416666666666666
        t1 = t + 0.010416666666666666
    elif quality == 2: 
        continue 
        t0 = t - 0.020833333333333332
        t1 = t + 0.020833333333333332
    elif quality == 3:
        t0 = t
        t1 = t + 0.041667
    elif quality == 4: 
        t0 = t
        t1 = t + 1
        
    event_time, events = satellite.find_events(station, t0, t1, altitude_degrees=15.0)
    event_time = event_time.utc_strftime('%Y %b %d %H:%M:%S')

    result_dir.append(alouette.loc[i, "Directory"])
    result_sub_dir.append(alouette.loc[i, "Subdirectory"])
    result_name.append(alouette.loc[i, "filename"])

    result_type.append(meta_type)
    
    result_read_epoch.append(record_epoch)
    result_read_station.append(alouette.loc[i, "Station_Name"])
    result_tle_epoch.append(nearest_epoch)

    result_flag.append(flag)

    result_events.append(events)
    result_event_times.append(event_time)

    result_alt.append(alt.degrees)
    result_distance.append(distance.km)

    result_lat.append(lat.degrees)
    result_lon.append(lon.degrees)
    result_height.append(height.km)

    clear_output(wait=True)
    print(i)
    
print("Procsesing Complete")

dict = {'Directory': result_dir, 'Subdirectory': result_sub_dir, 'filename': result_name, 'processed_type': result_type,
        'Read_Epoch': result_read_epoch, 'Read_Station': result_read_station, 'TLE_Epoch': result_tle_epoch,
        'Flag': result_flag, 'Events': result_events, 'Event_Times': result_event_times,
        'Station_Alt': result_alt, 'Station_Distance': result_distance,
        'Sat_Lat': result_lat, 'Sat_Lon': result_lon, 'Sat_Height': result_height} 
        
        
df = pd.DataFrame(dict)
     
# saving the dataframe
df.to_csv('results_full_run.csv')